# Importing stuff:

In [ ]:
!pip install googletrans
!pip install indic_transliteration
!pip install praw

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import googletrans
from googletrans import Translator
trans = Translator()

import praw
import re

# HinGerman Dataset

Grabbing some hinglish offensive tweets here:

In [ ]:
#!wget https://raw.githubusercontent.com/ramitsawhney27/OffensiveHinglishTweetClassification/master/HOT_Dataset_modified.csv

--2020-10-10 11:15:14--  https://raw.githubusercontent.com/ramitsawhney27/OffensiveHinglishTweetClassification/master/HOT_Dataset_modified.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 508576 (497K) [text/plain]
Saving to: ‘HOT_Dataset_modified.csv.1’

HOT_Dataset_modifie 100%[===================>] 496.66K  --.-KB/s    in 0.07s   

2020-10-10 11:15:14 (7.23 MB/s) - ‘HOT_Dataset_modified.csv.1’ saved [508576/508576]



In [ ]:
hot = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Abuse/MultiLingual/HOT.csv") # Reading the csv

In [ ]:
# converting their values in the isoffensive column to strings.

for i in hot[hot['isoffensive'] == 1.].index:  
  hot['isoffensive'].loc[i] = 'Yes' 

for i in hot[hot['isoffensive'] == 2.].index:
  hot['isoffensive'].loc[i] = 'Yes'

for i in hot[hot['isoffensive'] == 0.].index:
  hot['isoffensive'].loc[i] = 'No'


In [ ]:
hot.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
hot.columns = ['isoffensive', 'text']

In [ ]:
#hot.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/hinglish.csv")
# exporting it.

This is from the GermanEval18 competetion:
https://www.researchgate.net/publication/327914386_Overview_of_the_GermEval_2018_Shared_Task_on_the_Identification_of_Offensive_Language

In [ ]:
# Reading and taking a look at the data.
german = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/german/german.csv")
german 

,Unnamed: 0,text,isoffensive
0,0,user sie haben ja auch recht. unser tweet war ...,No
1,1,user fröhlicher gruß aus der schönsten stadt d...,No
2,2,user amis hätten alles und jeden gewählt...nur...,No
3,3,user kein verläßlicher verhandlungspartner. na...,Yes
4,4,"user ja, aber wo widersprechen die zahlen denn...",No
...,...,...,...
5003,5003,gegens. zul. zu patenamt amp gegenseitige aner...,No
5004,5004,"user zu merkel fällt mir nur ein, ein mal verr...",Yes
5005,5005,user user ein richtiges zeichen unserer nachba...,Yes
5006,5006,"user ,honecker‘merkel macht uns zur ,ddr‘ klag...",Yes


In [ ]:
german.drop(['Unnamed: 0'], axis = 1, inplace = True)  # Dropping an unnecessary column.
german = pd.concat([german['isoffensive'], german['text']], axis = 1)

In [ ]:
german.columns = ['text', 'isoffensive', 0] # Changing column names.
german.drop([0], axis = 1, inplace = True)  # Dropping an unnecessary column.

In [ ]:
# Converting the values in the 'isoffensive' column again:

for i in german[german['isoffensive'] == 'OTHER'].index:
  german['isoffensive'].loc[i] = 'No'

for i in german[german['isoffensive'] == 'OFFENSE'].index:
  german['isoffensive'].loc[i] = 'Yes'

I created this function to deal with tweets and it generalises pretty well for tweets in virtually any languages I have worked with.

In [ ]:
import re

def preprocess(text):
  l = re.sub(r'&', '', text)
  m = re.sub(r'RT ', '', l)
  n = re.sub(r'~', '', m)
  o = re.sub(r'#', '', n)
  p = re.sub(r'!+', '', o)
  q = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'link', p)
  r = re.sub(r'[*]', '', q)
  s = re.sub(r'[@]\w+', 'user', r)
  t = re.sub(r'[:|;]', '', s)
  u = re.sub(r'[\\x]\W+|\d', '', t)
  v = re.sub(r'[\\x]\W+|\d', '', u)
  w = re.sub(r'[\\x]\w+', '', v)
  x = re.sub(r'    ', ' ', w)
  y = re.sub(r'   ', ' ', x)
  z = re.sub(r'  ', ' ', y)
  ProcessedTweet = z.lower()
  return ProcessedTweet

In [ ]:
# Preprocessing the text:

for i in german.index:   
  german['text'].loc[i] = preprocess(german['text'].loc[i])

In [ ]:
#german.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/german.csv")

Grabbing the hinglish data again to use it for the three + language model:

In [ ]:
# Dropping a columns:
hot = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/hinglish/hinglish.csv")
hot.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
# Redefining column names:
hot.columns = ['isoffensive', 'text']

About a 1000 in hinglish are non abusive/ 2000+ are abusive.

In [ ]:
# Getting the german data and compiling:
german = pd.concat([german['isoffensive'], german['text']], axis = 1)

In [ ]:
# Finally our first dataset:

hingerman = pd.concat([german, hot], axis = 0)

size = hingerman.shape[0]
hingerman = hingerman.sample(n = size)
hingerman.index = range(size)

Our german dataset here has like a 1100 something 'Yes' and all others are 'No' which is giving me trouble.

In [ ]:
hingerman

,isoffensive,text
0,No,user why is nirodh nirdosh trending? has someo...
1,No,"hunde, die bellen, beißen auch. und wenn dich,..."
2,Yes,doitsche kauft nicht bai türken
3,Yes,user user user user user user user user user u...
4,Yes,so wie firmen zu groß werden können und ihre z...
...,...,...
8191,No,".user tweet tut uns leid. wir nahmen an, die i..."
8192,No,user user user diese menschen leiden an realit...
8193,Yes,user user user sie warnen vor nazis und führen...
8194,Yes,land mat chus gaandu mulle


In [ ]:
#hingerman.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/input/hingerman.csv")

In [ ]:
valid = hingerman[-2000:]
train = hingerman[6196:]
#valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/hingerman/valid.csv")
#train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/hingerman/train.csv")

# Exported it.

## English Text Data

In [ ]:
# Preprocessing stuff:

eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
eng = eng.sample(n = 6000)
eng.index = range(eng.shape[0])

# Grabbing more data for later use:

eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
ENGLISH = eng.sample(n = 10000)
ENGLISH.index = range(ENGLISH.shape[0])

In [ ]:
ENGLISH.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/english/ENGLISH.csv")

## German Text Data

Grabbing more german data:

In [ ]:
!wget https://hasocfire.github.io/hasoc/2019/files/german_dataset.zip

In [ ]:
pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/german/german.csv")

,Unnamed: 0,text,isoffensive
0,0,user sie haben ja auch recht. unser tweet war ...,No
1,1,user fröhlicher gruß aus der schönsten stadt d...,No
2,2,user amis hätten alles und jeden gewählt...nur...,No
3,3,user kein verläßlicher verhandlungspartner. na...,Yes
4,4,"user ja, aber wo widersprechen die zahlen denn...",No
...,...,...,...
5003,5003,gegens. zul. zu patenamt amp gegenseitige aner...,No
5004,5004,"user zu merkel fällt mir nur ein, ein mal verr...",Yes
5005,5005,user user ein richtiges zeichen unserer nachba...,Yes
5006,5006,"user ,honecker‘merkel macht uns zur ,ddr‘ klag...",Yes


In [ ]:
german2 = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/german/gg ds.csv", sep = '\t')
german2

,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE
...,...,...,...,...
3814,hasoc_de_3815,"akquirieren Männer,die sich um die Kinder kümm...",NOT,NONE
3815,hasoc_de_3816,"Ja,schon seit Jahren! Ich muß dran denken,das...",NOT,NONE
3816,hasoc_de_3817,@Kurkamp @wendt_joachim @Schroeder_Live @lamni...,NOT,NONE
3817,hasoc_de_3818,EU Wahlen! AfD für Kindererziehungszeiten für...,NOT,NONE


I think after this we'd have 9000 + german examples in our hingerman dataset.

In [ ]:
# Preprocessing it:

german2.drop(['task_2', 'text_id'], axis = 1, inplace = True) 

german2.columns = ['text', 'isoffensive']

german2 = pd.concat([german2['isoffensive'], german2['text']], axis = 1)

In [ ]:
for i in german2[german2['isoffensive'] == 'NOT'].index:     # 3412 rows of not offensive text here
  german2['isoffensive'].loc[i] = 'No'  

for i in german2[german2['isoffensive'] == 'HOF'].index:     # 407 rows of not offensive text here
  german2['isoffensive'].loc[i] = 'Yes'          

In [ ]:
for i in german2.index:
  german2['text'].loc[i] = preprocess(german2['text'].loc[i])

Okay till now we have about 1500+ abusive german examples and 7000+ non-abusive.

This is  my attempt to convert english data into german for the model to generalise better.

In [ ]:
eng[eng['isoffensive'] == 'Yes'] 

,isoffensive,text
0,Yes,and my bitch so bad that i'm never ever cheating
1,Yes,user only faggots drive stick
2,Yes,user blow shit up and get fatter. white trash ...
4,Yes,user user user you're a faggotyou're a te fan...
6,Yes,user how many fuckin fags did a even get? shou...
...,...,...
5993,Yes,i be chillin wit the bitches cuz a lotta nicca...
5994,Yes,lmfaooo user whenever you take selfies at work...
5996,Yes,"user ""user shes one of the ugliest women ive e..."
5997,Yes,i just said that user my tl gotta chill with ...


point to be noted: about 4000 abusive text rows, 2000 clean.

In [ ]:
german3 = eng.copy()
for i in german3.index:
  text = german3.text.loc[i]
  german = (trans.translate(text, dest = 'de'))
  german3.text.loc[i] = german.text


In [ ]:
german3.to_csv('/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/GERMAN.csv')

In [ ]:
GERMAN = pd.concat([german3, german2, german], axis = 0)
GERMAN.to_csv('/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/german/GERMAN.csv')

Now in the german language we have about 5500+ abusive row and the non-abusive ones are up to 9000+

## Hindi Text Data

In [ ]:
hind = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Abuse/MultiLingual/hindi_dataset.csv", sep = '\t')

In [ ]:
hind = pd.concat([hind['task_1'], hind['text']], axis = 1)
hind.columns = ['isoffensive', 'text']

In [ ]:
NOT = hind[hind['isoffensive'] == 'NOT'].index

HOF = hind[hind['isoffensive'] == 'HOF'].index


In [ ]:
for i in HOF:
  hind['isoffensive'].loc[i] = 'Yes'

for i in NOT:
  hind['isoffensive'].loc[i] = 'No'

In [ ]:
from indic_transliteration import sanscript, detect
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

def translit(text):
  newtext = transliterate(text, sanscript.DEVANAGARI, sanscript.IAST)
  return newtext.lower()

In [ ]:
translit('क्या है तुझे')  # Does it work? 

'kyā hai tujhe'

In [ ]:
for i in hind.index:
  hind['text'].loc[i] = translit(hind['text'].loc[i])

In [ ]:
import re

def hindiprepro(text):
  a = re.sub('ī', 'i', text)
  b = re.sub('ā', 'a', a)
  c = re.sub('ṃ', 'n', b)
  d = re.sub('ś', 'sh', c)
  e = re.sub('|', '', d)
  f = re.sub('cā', 'ka', e)
  g = re.sub('  ', ' ', f)
  h = re.sub('#', '', g)
  i = re.sub('ū', 'u', h)
  j = re.sub('ḍ', 'd', i)
  k = re.sub('a़', 'a', j)
  l = re.sub('ṭ', 't', k)
  m = re.sub('-', '', l)
  n = re.sub('c', 'ch', m)
  o = re.sub('ṣaṇa', 'sh', n)
  return o.lower()

In [ ]:
for i in hind.index:
  hind['text'].loc[i] = hindiprepro(hind['text'].loc[i])

In [ ]:
for i in hind.index:
  hind['text'].loc[i] = preprocess(hind['text'].loc[i])

In [ ]:
hind[hind['isoffensive'] == 'No']

In [ ]:
hind.text[7]

In [ ]:
eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
hind2 = eng.sample(n = 6000)
hind2.index = range(hind2.shape[0])

In [ ]:
for i in hind2.index:
  text = hind2['text'].loc[i]
  newtext = (trans.translate(text, dest = 'hi')).text
  hind2['text'].loc[i] = newtext

In [ ]:
for i in hind2.index:
  hind2['text'].loc[i] = translit(hind2['text'].loc[i])

In [ ]:
for i in hind2.index:
  hind2['text'].loc[i] = hindiprepro(hind2['text'].loc[i])

In [ ]:
hind2

,isoffensive,text
0,Yes,sabatavitatavitaveta kyuja skini nigaha lekina...
1,Yes,upayogakarta ko khushi hui ki apa usa bachche ...
2,Yes,upayogakarta upayogakarta apake tebo ke satha ...
3,Yes,user dis hoe bri sapane men eka gile sapane ko...
4,Yes,jaba taka ne rednechks jaba taka main apako ph...
...,...,...
5995,Yes,ja raha hai mushkila kutiya kasama khata hum̐ ...
5996,Yes,upayogakarta ne apako bataya ki ve kutiya paga...
5997,No,i heard this when i was only 6 years old and i...
5998,Yes,upayogakarta ina hoes vaphadara nahin hai


In [ ]:
hind = pd.concat([hind, hot, hind2], axis = 0)
hind.index = range(hind.shape[0])

In [ ]:
hind.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")

Finally integrating about 2000 abusive hinglish text rows and 2000 non-abusive ones. Getting the total examples to 7000+ out of which 3000 are non-abusive while 4000+ are abusive.

## Compiling EngHinGerman

In [ ]:
enghingerman = pd.concat([GERMAN, hind, ENGLISH], axis = 0)

enghingerman.index = range(enghingerman.shape[0])
enghingerman.shape

(32680, 2)

In [ ]:
train = enghingerman[:25000]
valid = enghingerman[-7680:]

train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/eng-hin-german #2/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/eng-hin-german #2/valid.csv")
enghingerman.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/eng-hin-german #2/enghingerman.csv")


In [ ]:
enghingerman.shape

(32680, 2)

# FrenEngHinGerman

## French

In [ ]:
eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']


french = eng.sample(n = 6000)          # Change 'n' as you further train
french.index = range(french.shape[0])

In [ ]:
from googletrans import Translator
trans = Translator()

(trans.translate('i like today', dest = 'fr')).text

"j'aime aujourd'hui"

In [ ]:
for i in french.index:
  text = french['text'].loc[i]
  newtext = (trans.translate(text, dest = 'fr')).text
  french['text'].loc[i] = newtext

In [ ]:
french.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/french.csv")

## Compiling for FrenEngHinGerman

In [ ]:
german = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/GERMAN.csv")
french = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/french.csv")
hindi = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")
english = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/ENGLISH.csv")

In [ ]:
input = pd.concat([german, english, hindi, french], axis = 0)

In [ ]:
input = input.sample(n = input.shape[0])
input.index = range(input.shape[0])

In [ ]:
input


,Unnamed: 0,isoffensive,text,Unnamed: 0.1
0,6357,Yes,damaadji saale gaandu,6357.0
1,1833,No,everyones back lool this is almost 3 years old...,NaN
2,11204,Yes,mujhe pata hai ki agara main nae sala taka nah...,NaN
3,5713,Yes,user oomf pussy probably taste like the krabby...,NaN
4,3319,Yes,Benutzer diese Hündin hat die iTunes-Nutzungsb...,NaN
...,...,...,...,...
35848,13403,Yes,aba apa behosha hone taka pi rahe hain| mujhe ...,NaN
35849,5266,Yes,ima start friend zoning these hoes,NaN
35850,5795,Yes,user this bitch just blew my high and i was hi...,NaN
35851,778,No,"user even shorter charlie sheen ""your move, me...",NaN


In [ ]:
train = input[:28000]
valid = input[-7853:]

train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FrenEngHinGerman #3/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FrenEngHinGerman #3/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FrenEngHinGerman #3/input.csv")


# SpanFrenEngHinGer Model a.

## Spanish

In [ ]:
eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
span = eng.sample(n = 6000)                      # Play around with the n.
span.index = range(span.shape[0])

In [ ]:
for i in span.index:
  text = span['text'].loc[i]
  newtext = (trans.translate(text, src = 'en', dest = 'es')).text
  span['text'].loc[i] = newtext

In [ ]:
span.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
span.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/SPANISH.csv")

## Hindi

In [ ]:
eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
hind3 = eng.sample(n = 10000)                      # Play around with the n.
hind3.index = range(hind2.shape[0])

In [ ]:
for i in hind3.index:
  text = hind3['text'].loc[i]
  newtext = (trans.translate(text, src = 'en', dest = 'hi')).text
  hind3['text'].loc[i] = newtext

In [ ]:
for i in hind3.index:
  hind3['text'].loc[i] = translit(hind3['text'].loc[i])

In [ ]:
for i in hind3.index:
  hind3['text'].loc[i] = hindiprepro(hind3['text'].loc[i])

In [ ]:
hot = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/hinglish/hinglish.csv")
hind = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/hinglish/HIND.csv")
hot.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
hind = pd.concat([hind, hot, hind3], axis = 0)
hind.index = range(hind.shape[0])                 ## pls fix dis

In [ ]:
hind.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")   # Here's where you grab the total data

## Compiling for SpanFrenEngHinGerman

In [ ]:
german = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/GERMAN.csv")
french = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/french.csv")
hindi = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")
english = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/ENGLISH.csv")
spanish = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/SPANISH.csv")

In [ ]:
french.drop(['Unnamed: 0'], axis = 1, inplace = True)
german.drop(['Unnamed: 0'], axis = 1, inplace = True)
english.drop(['Unnamed: 0'], axis = 1, inplace = True)
spanish.drop(['Unnamed: 0'], axis = 1, inplace = True)
hindi.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)


In [ ]:
input = pd.concat([german, english, hindi, french, spanish], axis = 0)

In [ ]:
input = input.sample(n = input.shape[0])
input.index = range(input.shape[0])

In [ ]:
input

,isoffensive,text
0,Yes,"yo perro azadas no una, sino todas las azadas"
1,Yes,user you will and he's gonna get all dem bitches
2,No,india chaptain virat kohli bangladesh has play...
3,Yes,abhisara sara eka bhafava hai jo jaham̐ roti m...
4,No,user user user would not spend my money on tra...
...,...,...
50675,Yes,now i seperate from niggas i don't trust nigga...
50676,Yes,user can someone show hailie this tweet? bitch...
50677,Yes,user ya dude get the pussies drunk
50678,No,but pls dont play in others life.


The dataset above has about 20000+ non-abusive texts, while it also has 30000+ abusive texts.

In [ ]:
train = input[:40680]
valid = input[-10000:]

train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/SpanFrenEngHinGerman #3/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/SpanFrenEngHinGerman #3/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/SpanFrenEngHinGerman #3/input.csv")


# TamSpan

---

Tamil needs improvement

##Tamil 

In [ ]:
eng = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model #11 English/train.csv")
eng.drop(['Unnamed: 0'], axis = 1, inplace = True)
eng.columns = ['isoffensive', 'text']
tamil = eng.loc[:1000]
tamil2 = eng.loc[1000:2000]
tamil3 = eng.loc[2000:3000]
tamil4 = eng.loc[3000:4000]
tamil5 = eng.loc[4000:5000]
tamil6 = eng.loc[5000:6000]
tamil7 = eng.loc[6000:7000]
tamil8 = eng.loc[7000:8000]
tamil9 = eng.loc[8000:9000]
tamil.index = range(tamil.shape[0])

In [ ]:
for i in tamil.index:
  text = tamil['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil['text'].loc[i] = newtext.text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
tamil2 = tamil2.dropna()

In [ ]:
for i in tamil2.index:
  text = tamil2['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil2['text'].loc[i] = newtext.text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
tamil3 = tamil3.dropna()

for i in tamil3.index:
  text = tamil3['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil3['text'].loc[i] = newtext.text

In [ ]:
tamil4 = tamil4.dropna()
tamil5 = tamil5.dropna()
tamil6 = tamil6.dropna()
tamil7 = tamil7.dropna()
tamil8 = tamil8.dropna()
tamil9 = tamil9.dropna()


In [ ]:
for i in tamil4.index:
  text = tamil4['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil4['text'].loc[i] = newtext.text

In [ ]:
for i in tamil5.index:
  text = tamil5['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil5['text'].loc[i] = newtext.text

In [ ]:
for i in tamil6.index:
  text = tamil6['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil6['text'].loc[i] = newtext.text

In [ ]:
for i in tamil7.index:
  text = tamil7['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil7['text'].loc[i] = newtext.text

In [ ]:
for i in tamil8.index:
  text = tamil8['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil8['text'].loc[i] = newtext.text

In [ ]:
for i in tamil9.index:
  text = tamil9['text'].loc[i]
  newtext = trans.translate(text, src = 'en', dest = 'ta')
  tamil9['text'].loc[i] = newtext.text

In [ ]:
tamil = pd.concat([tamil, tamil2, tamil3, tamil4, tamil5, tamil6, tamil7, tamil8, tamil9], axis = 0)

In [ ]:
tamil.index = range(tamil.shape[0])

In [ ]:
from indic_transliteration import sanscript, detect
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

def translittamil(text):
  newtext = transliterate(text, sanscript.TAMIL, sanscript.IAST)
  return newtext.lower()

In [ ]:
originaltamil = tamil.copy()

for i in tamil.index:
  tamil['text'].loc[i] = translittamil(tamil['text'].loc[i])

In [ ]:
import re

def hindiprepro(text):
  a = re.sub('ī', 'i', text)
  b = re.sub('ā', 'a', a)
  c = re.sub('ṃ', 'n', b)
  d = re.sub('ś', 'sh', c)
  e = re.sub('|', '', d)
  f = re.sub('cā', 'ka', e)
  g = re.sub('  ', ' ', f)
  h = re.sub('#', '', g)
  i = re.sub('ū', 'u', h)
  j = re.sub('ḍ', 'd', i)
  k = re.sub('a़', 'a', j)
  l = re.sub('ṭ', 't', k)
  m = re.sub('-', '', l)
  n = re.sub('c', 'ch', m)
  o = re.sub('ṣaṇa', 'sh', n)
  return o.lower()

In [ ]:
for i in tamil.index:
  tamil['text'].loc[i] = hindiprepro(tamil['text'].loc[i])

In [ ]:
tamil

,isoffensive,text
0,Yes,user follow my dog.. he has over k bitchhes us...
1,Yes,mighavun oriṉa jherghghaiyaḻaragha iṛndha bhay...
2,Yes,alibhabha / jagh ma ghodhdha ôṛ bhaiyaṉai nejh...
3,No,user user user it's a mutation. genealogy is c...
4,Yes,ladhdhari bhoṉr̂a indha maṇvêdhdhighaḻai viḻai...
...,...,...
9003,No,user mochk draft?
9004,Yes,êṉ dhôlaibhejhi eṉ ôṛ bhijh
9005,Yes,êṉ bhidhjhughaḻiliṛndhu nirvaṇaṅghaḻaibh bhêr̂...
9006,Yes,bhayaṉar hus indha nadhghaḻil iṇaibhbhu bhoṉr̂...


In [ ]:
tamil.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/tamil.csv")

## Compiling for TamSpan

In [ ]:
german = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/GERMAN.csv")
french = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/french.csv")
hindi = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")
english = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/ENGLISH.csv")
spanish = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/SPANISH.csv")
tamil = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/tamil.csv")

In [ ]:
french.drop(['Unnamed: 0'], axis = 1, inplace = True)
german.drop(['Unnamed: 0'], axis = 1, inplace = True)
english.drop(['Unnamed: 0'], axis = 1, inplace = True)
spanish.drop(['Unnamed: 0'], axis = 1, inplace = True)
hindi.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
tamil.drop(['Unnamed: 0'], axis = 1, inplace = True)


In [ ]:
input = pd.concat([german, english, hindi, french, spanish, tamil], axis = 0)

In [ ]:
input = input.sample(n = input.shape[0])
input.index = range(input.shape[0])

In [ ]:
input = input.dropna()

In [ ]:
input

,isoffensive,text
0,No,"eka aura muslima shakhsa ka moba linchinga, jh..."
1,No,apane pita daॉ vakaara ahamada shaha sahaba ke...
2,No,user i drowning in my own tears link
3,Yes,"' if this ugly ass niggah can be with her , i ..."
4,Yes,bhayaṉar niṅghaḻ bhughaibhidhighghun
...,...,...
59683,Yes,user das ist die stadt mit der armlänge.... lb...
59684,No,apana dila apani tabahi ka sababa hota hai.. y...
59685,Yes,user i'm convinced that some of y'all bitches ...
59686,Yes,sacha sunate hi kutte ki taraha duma dabakara ...


In [ ]:
train = input[:50000]
valid = input[-9682:]

train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/TanSpanFrenEngHinGer #5/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/TanSpanFrenEngHinGer #5/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/TanSpanFrenEngHinGer #5/input.csv")


## Korean
---
Got this data through the folowing repository: https://github.com/kocohub/korean-hate-speech/tree/master/labeled

This model will be able to predict on Tamil, Spanish, French, English, Hinglish and German.

In [ ]:
!pip install korean_romanizer

In [ ]:
!wget https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv

korean = pd.read_csv("/content/train.tsv", sep= '\t')
korean = pd.concat([korean['hate'], korean['comments']], axis = 1)

In [ ]:
for i in korean[korean['hate'] == 'hate'].index:
  korean.hate.loc[i] = 'Yes'

for i in korean[korean['hate'] == 'none'].index:
  korean.hate.loc[i] = 'No'  

In [ ]:
from korean_romanizer.romanizer import Romanizer

def kor2roman(text):
  r = Romanizer(text)
  roman = r.romanize() 
  return roman


In [ ]:
for i in korean.index

#Final Model a.

## French

## Spanish

In [ ]:
cleanspanish = convert2(eng2, eng2.index, 'es')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
def addnewdata(nodataset, yesdataset):
  
  yesdata = yesdataset[yesdataset['isoffensive'] == 'Yes']

  data = pd.concat([nodataset, yesdata], axis = 0)
  data = data.sample(n = data.shape[0])
  data.index = range(data.shape[0])
  return data

In [ ]:
spanish = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/spanish.csv")
spanish.drop(['Unnamed: 0'], axis = 1, inplace = True)
spanish = addnewdata(cleanspanish, spanish)
spanish.shape

(11964, 3)

In [ ]:
spanish.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/malayalam.csv")

## English


In [ ]:
reddit = praw.Reddit(client_id = 'bpbCjQVOP9_6_w', client_secret = '8fG58nA_SmtIZTiRk_lyAHuSKmQ',
                     user_agent = 'GlueLabs')

In [ ]:
# Defining a function that gets text from reddit and provides us with a DataFrame:
def getred(text):
  subreddit = []
  posts = reddit.subreddit(text).top(limit = 1000)
  for post in posts:
    subreddit.append(post.title)

  # Producing a list of annotations:
  isoffensive = ['No']*1000
  
  # Making the DataFrames that will be concatenated:
  subreddit = pd.DataFrame(subreddit, columns = ['text'])
  isoffensive = pd.DataFrame(isoffensive, columns = ['isoffensive'])

  data = pd.concat([isoffensive, subreddit], axis = 1)
  data = data.dropna()
  return data

In [ ]:
business = getred('Business')

In [ ]:
entrepreneur = getred('Entrepreneur')

In [ ]:
LPT = getred('LifeProTips')
for i in LPT.index:
  LPT.text.loc[i] = re.sub('LPT: ', '', LPT.text.loc[i])

In [ ]:
cleanjokes = getred('CleanJokes')

In [ ]:
startup = getred('startups')

In [ ]:
sb = getred('smallbusiness')

In [ ]:
sb.shape

(998, 2)

In [ ]:
econ = getred('Economics')

In [ ]:
iwtl = getred('IWantToLearn')

for i in iwtl.index:
  iwtl.text.loc[i] = re.sub("IWTL", "", iwtl.text.loc[i])

In [ ]:
# Combining the DataFrames to form a major one:
eng2 = pd.concat([iwtl, econ, sb, startup, cleanjokes, LPT, entrepreneur, business], axis = 0)

In [ ]:
#eng2.to_csv("/content/drive/My Drive/Glue Labs/Data-Abuse/eng2.csv")
eng2 = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Abuse/eng2.csv")

In [ ]:
def convert2(dataset, index, dest):
  for i in index:
    text = dataset.text.loc[i]
    translation = trans.translate(text, dest = dest)
    dataset.text.loc[i] = translation.pronunciation
  return dataset

In [ ]:
cleanhindi = convert2(eng2, eng2.index, 'hi')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
cleanhindi

,Unnamed: 0,isoffensive,text
0,0,No,yukti: yadi aap nahin jaanate ki kahaan se shu...
1,1,No,main seekhana chaahata hoon kee mahilaon ke aa...
2,2,No,75 kaurase sarteephiket jo ab aap mupht mein k...
3,3,No,"namaste vahaan, agar kisee kee dilachaspee (ki..."
4,4,No,[[]]
...,...,...,...
7909,984,No,arabapati ron perelamain lagabhag har cheej ko...
7910,985,No,sveet tomaito koronovaayaras shatadaun par har...
7911,986,No,", khudara sarvanaash ’jaaree hai: gaip, phaimi..."
7912,987,No,Big Pharma companies spend more on marketing t...


In [ ]:
cleanhindi.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/Clean Translates/hindi.csv")

In [ ]:
cleankorean = convert2(eng2, eng2.index, 'ko')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
cleankorean


,Unnamed: 0,isoffensive,text
0,0,No,tib : eodiseobuteo sijaghaeyahalji moleugessda...
1,1,No,yeoseongdeul-ege jeogjeolhage ilhaneun beob-eu...
2,2,No,ije mulyolo hoegdeug hal su-issneun 75 gaeui k...
3,3,No,"annyeonghaseyo, nugungaga gwansim-i issdamyeon..."
4,4,No,[[]]
...,...,...,...
7909,984,No,Billionaire Ron Perelman selling almost everyt...
7910,985,No,Sweet Tomatoes confirms it’s permanently closi...
7911,986,No,"'somae jongmal'gyesog : Gap, Family Dollar, su..."
7912,987,No,daehyeong jeyag hoesaneun R & Dboda maketing-e...


In [ ]:
cleankorean.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/Clean Translates/korean.csv")


In [ ]:
cleantelugu = convert2(eng2, eng2.index, 'te')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
cleantelugu

,Unnamed: 0,isoffensive,text
0,0,No,Ciṭkā: Ekkaḍa prārambhin̄cālō mīku teliyakapōt...
1,1,No,Nēnu mahiḷala cuṭṭū elā pani cēyālō nērcukōvāl...
2,2,No,75 Kōrsula sarṭiphikēṭ mīru ippuḍu ucitaṅgā sa...
3,3,No,"Halō, evarainā āsakti kaligi uṇṭē (konni kāraṇ..."
4,4,No,[[]]
...,...,...,...
7909,984,No,Billionaire Ron Perelman selling almost everyt...
7910,985,No,Karōnāvairas ṣaṭḍaunlapai prati sthānānni śāśv...
7911,986,No,"‘Retail apocalypse’ continues: Gap, Family Dol..."
7912,987,No,Big phārmā kampenīlu ār‌aṇḍ‌ḍi kaṇṭē mārkeṭiṅg...


In [ ]:
cleantelugu.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/Clean Translates/korean.csv")

In [ ]:
new

## Malayalam

In [ ]:
def addnewdata(nodataset, yesdataset):
  
  yesdata = yesdataset[yesdataset['isoffensive'] == 'Yes']

  data = pd.concat([nodataset, yesdata], axis = 0)
  data = data.sample(n = data.shape[0])
  data.index = range(data.shape[0])
  return data

In [ ]:
malayalam = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/malayalam.csv")
malayalam.drop(['Unnamed: 0'], axis = 1, inplace = True)
malayalam = addnewdata(cleanmalayalam, malayalam)
malayalam.shape

(13299, 2)

In [ ]:
malayalam.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/malayalam.csv")

## Chinese

In [ ]:
chin = eng.loc[:8000]
chin = chin.dropna()

In [ ]:
for i in chin.index:
  text = chin.text.loc[i]
  chinese = trans.translate(text, src = 'en', dest = 'zh-cn')
  chin.text.loc[i] = chinese.texta

In [ ]:
for i in chin.index:
  text = chin.text.loc[i]
  chinese = trans.translate(text, src = 'en', dest = 'zh-cn')
  chin.text.loc[i] = chinese.pronunciation

In [ ]:
chin

,isoffensive,text
0,Yes,用户跟随我的狗..他有超过k个母狗用户我今天在公园里为我的母狗做了一个金毛寻回犬。链接...
1,Yes,user that was so gay. and i was tanning bitch....
2,Yes,阿里巴巴/杰克·马·达塔（Alibaba / Jack Ma Gotta）爱一个击败梅格·惠...
3,No,用户用户用户这是一个突变。家谱是令人费解的。基本上，宰杀了斑马基因
4,Yes,像彩票一样玩这些ho头
...,...,...
7996,Yes,Yònghù kàn qǐlái xiàng shì yào chuīxū mǒu rén
7997,Yes,"""user user user bitch fuck u link maybe you'll..."
7998,No,"only just got this message, not ignoring you. ..."
7999,Yes,you fucking fag user this is e what i am talki...


In [ ]:
chin.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/chinese.csv")

## Korean

In [ ]:
cleankorean = convert2(eng2, eng2.index, 'ko')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
cleankorean.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/Clean Translates/korean.csv")


In [ ]:
def addnewdata(nodataset, yesdataset):
  
  yesdata = yesdataset[yesdataset['isoffensive'] == 'Yes']

  data = pd.concat([nodataset, yesdata], axis = 0)
  data = data.sample(n = data.shape[0])
  data.index = range(data.shape[0])
  return data

In [ ]:
malayalam = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/korean.csv")
malayalam.drop(['Unnamed: 0'], axis = 1, inplace = True)
malayalam = addnewdata(cleanmalayalam, malayalam)
malayalam.shape

(13299, 2)

In [ ]:
malayalam.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/malayalam.csv")

## Telugu

In [ ]:
tel = eng.loc[:6000]
tel = tel.dropna()

In [ ]:
for i in tel.index:
  text = tel.text.loc[i]
  telugu = trans.translate(text, src = 'en', dest = 'te')
  tel.text.loc[i] = telugu.pronunciation

## Final Compile a.

In [ ]:
german = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/GERMAN.csv")
french = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/french.csv")
hindi = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/HINDI.csv")
english = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/ENGLISH.csv")
spanish = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/SPANISH.csv")
tamil = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/tamil.csv")
malayalam = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/malayalam.csv")
telugu = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/telugu.csv")
chinese = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/chinese.csv")
korean = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/inputs/korean.csv")


In [ ]:

french.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
german.drop(['Unnamed: 0'], axis = 1, inplace = True)
english.drop(['Unnamed: 0'], axis = 1, inplace = True)
spanish.drop(['Unnamed: 0'], axis = 1, inplace = True)
hindi.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
malayalam.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
telugu.drop(['Unnamed: 0'], axis = 1, inplace = True)
chinese.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1, inplace = True)
korean.drop(['Unnamed: 0'], axis = 1, inplace = True)
tamil.drop(['Unnamed: 0'], axis = 1, inplace = True)


In [ ]:
input = pd.concat([german, english, hindi, french, spanish, tamil, telugu, korean, malayalam, chinese], axis = 0)

In [ ]:
input = input.sample(n = input.shape[0])
input = input.dropna()
input.index = range(input.shape[0])

In [ ]:
for i in input.index:
  input.text.loc[i] = input.text.loc[i].lower()

In [ ]:
input

,isoffensive,text
0,No,girl scout sells hundreds of cookie boxes outs...
1,No,aseupilin-i simjang mabi jung-e saengmyeong-eu...
2,No,about some more subreddits to learn things
3,Yes,딸도 죄가 있지. 악마의 씨앗에서 태어난것도 문제가 되고...
4,Yes,fremde iphone imma nennen einen goth hoe link
...,...,...
120319,No,[[]]
120320,Yes,a lotta u niccas ain't who u say u is
120321,Yes,user user wir müssen zulegen und merkel vernic...
120322,Yes,bhayaṉar adhaghghamagha indha umai sbhigh bhad...


In [ ]:
train = input[:100000]
valid = input[-20324:]

train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/input.csv")


In [ ]:
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/input.csv")


In [ ]:
reddit = pd.read_csv("/content/drive/My Drive/Glue Labs/Data-Abuse/reddit.csv")

In [ ]:
reddit

,id,text,hate_speech_idx,response
0,1. e8q18lf\n2. \te8q9w5s\n3. \t\te8qbobk\n4. \...,1. A subsection of retarded Hungarians? Ohh bo...,[1],"[""I don't see a reason why it's okay to insult..."
1,1. e9c6naz\n2. \te9d03a5\n3. \t\te9d8e4d\n,"1. > ""y'all hear sumn?"" by all means I live i...",[3],['Persons with disabilities is the accepted te...
2,1. e84rl2i\n2. \te84w60l\n3. \t\te8544rn\n4. \...,1. wouldn't the defenders or whatever they are...,NaN,NaN
3,1. e7kq72n\n2. \te7m24ar\n,1. Because the Japanese aren't retarded and kn...,[1],"[""It's not right for anyone of any gender to b..."
4,1. e7hdgoh\n2. \te7iyj6a\n3. \t\te7j6iho\n4. \...,1. That might be true if we didn't have an exa...,"[2, 3]","[""You shouldn't be bringing up sensitive topic..."
...,...,...,...,...
5015,1. e90fta8\n2. \te90g8px\n3. \t\te90jlq5\n4. \...,1. Who does she claim as *her people*?\n2. \tI...,[8],['its inappropriate to use the word faggot as ...
5016,1. e9b7mn2\n2. \te9bv6ir\n,1. Here's what's going to happen: Broward is ...,NaN,NaN
5017,1. e7q8ilv\n2. \te7qddqq\n3. \t\te7qstbc\n4. \...,1. Oh boy...so here comes a long venting post ...,"[1, 4]","[""using the word 'nigger' is a direct attack t..."
5018,1. dwp8jut\n,"1. OP, stop being a faggot and post videos nex...",[1],"[""using the words 'faggot fuckers' is offensiv..."


# Final Compile b

In [ ]:
input = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/input.csv")

In [ ]:
input = input.dropna()

In [ ]:
for i in input.index:
  nantext = re.sub('[\d+[a-z]+]', "N@N", input.text.loc[i])

  t = re.search('N@N', nantext)
  t = str(t)
  if t == 'None':
    nantext = numpy.NaN
  
  input.text.loc[i] = nantext

In [ ]:
import regex
def compileB(text):
  a = re.sub('-', ' ', text)
  b = re.sub('\n', ' ', a)
  c = re.sub('"', ' " ', b)

  d = re.sub('\$', '$ ', c)
  e = re.sub('£', ' ', d)

  f = re.sub('\(', '', e)
  g = re.sub('\)', '', f)
  h = re.sub('/', '/ ', g)
  i = re.sub('\[\[]]', '', h)
  j = re.sub(':', ' : ', i)
  k = re.sub('\...', ' ... ', j)




  x = re.sub('    ', ' ', k)
  y = re.sub('   ', ' ', x)
  z = re.sub('  ', ' ', y)
  return z

In [ ]:
import regex
def premodel(text):

  a = re.sub('-', ' ', text)
  c = re.sub('"', ' " ', b)

  d = re.sub('$', '$ ', c)
  e = re.sub('£', ' ', d)



  x = re.sub('    ', ' ', w)
  y = re.sub('   ', ' ', x)
  z = re.sub('  ', ' ', y)
  return e

In [ ]:
for i in input.index:
  t = input.text.loc[i]
  input.text.loc[i] = compileB(t)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
input

,isoffensive,text
0,No,i'll talk to the others and probably just come...
1,Yes,wenn ich täglich diesen geistigen dünnschiss ü...
2,Yes,"um eine brustverkleinerung zu bekommen, stehen..."
3,No,how do you make a tissue dance?
4,No,'it's really over' : corporate pensions head f...
...,...,...
115244,Yes,gijaya neodo ingan ini?
115245,Yes,user all i heard was i'm a bitch nigga
115246,No,gaurakṣa ke nama para bhifa ekatra karake kisi...
115247,No,merkel hau ab die mehrheit der deutschen wünsc...


In [ ]:
train = input[:100000]
valid = input[-15249:]
minival = input[5000:8000]

minival.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/minival.csv")
train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/input.csv")


# Final Compile c

In [ ]:
!git clone https://github.com/RobertJGabriel/Google-profanity-words.git
!git clone https://github.com/pmathur5k10/Hinglish-Offensive-Text-Classification.git
!git clone https://github.com/khvorostin/useful-english-phrases.git

fatal: destination path 'Google-profanity-words' already exists and is not an empty directory.
Cloning into 'Hinglish-Offensive-Text-Classification'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.


In [ ]:
abusivew = pd.read_csv("/content/Google-profanity-words/list.txt", sep='\n')
abusivew.columns = ['text']
list = pd.DataFrame(['Yes']*450, columns = ['isoffensive'])
eng_abuse = pd.concat([list, abusivew], axis = 1)


In [ ]:
# Adding some greetings in english:
eng = pd.read_csv("/content/useful-english-phrases/09-conversational-phrases.txt", sep='\n')
eng.columns = ['text']
list = pd.DataFrame(['Yes']*939, columns = ['isoffensive'])
eng1 = pd.concat([list, eng], axis = 1)

In [ ]:
hingab = pd.read_csv("/content/Hinglish-Offensive-Text-Classification/Hinglish_Profanity_List.csv", encoding= 'unicode_escape')
hingab.columns = ['text', 'isoffensive', '0']

for i in hingab.index:
  hingab.isoffensive.loc[i] = 'Yes'

hingab.drop(['0'], axis = 1, inplace = True)

hingab = pd.concat([hingab['isoffensive'], hingab['text']], axis = 1)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
input = pd.read_csv("/content/drive/My Drive/Glue Labs/Abuse Model 12/FINAL 10/input.csv")


In [ ]:
input.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
input = input.dropna()

In [ ]:
input = pd.concat([hingab, input, eng_abuse, eng1], axis = 0)

In [ ]:
import regex
def compileC(text):

  a = re.sub('\'', ' ', text)
  c = re.sub('"', '', a)

  x = re.sub('    ', ' ', a)
  y = re.sub('   ', ' ', x)
  z = re.sub('  ', ' ', y)
  return z

In [ ]:
input.shape

(115907, 2)

In [ ]:
input.index = range(input.shape[0])

In [ ]:
for i in input.index:
  input.text.loc[i] = compileC(input.text.loc[i])

In [ ]:
input = input.sample(n = input.shape[0])
input.index = range(input.shape[0])

In [ ]:
train = input[:100000]
valid = input[-15907:]
minival = input[5000:8000]

minival.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 13/input/minival.csv")
train.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 13/input/train.csv")
valid.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 13/input/valid.csv")
input.to_csv("/content/drive/My Drive/Glue Labs/Abuse Model 13/input/input.csv")